In [1]:
from datetime import datetime, timedelta
from sqlalchemy import create_engine, MetaData, Table
from scrape_sofa import get_sofascore_api_data, init_driver, get_all_players_stats_fbref, get_all_teams_stats_fbref
from funciones import get_stat
import pandas as pd

today = datetime.now()
yesterday = today - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
engine = create_engine('mysql+pymysql://root@localhost/footviz')
df_partidos_ayer = pd.read_sql(f"SELECT * FROM partido WHERE fecha = '{yesterday}'", engine)


In [3]:
df_partidos_ayer

,id_partido,jornada,url_fbref,url_sofascore,fecha,temporada,equipo_local,equipo_visitante,estadio,arbitro
0,31,4,https://fbref.com/en/matches/4fe3e679/Arsenal-...,https://www.sofascore.com/football/match/arsen...,2025-09-13,1,2,16,2,17.0
1,32,4,https://fbref.com/en/matches/8398bbef/Bournemo...,https://www.sofascore.com/football/match/bourn...,2025-09-13,1,1,5,1,18.0
2,33,4,https://fbref.com/en/matches/e3cade07/Crystal-...,https://www.sofascore.com/football/match/sunde...,2025-09-13,1,8,17,8,36.0
3,34,4,https://fbref.com/en/matches/a624dc89/Everton-...,https://www.sofascore.com/football/match/evert...,2025-09-13,1,9,3,9,19.0
4,35,4,https://fbref.com/en/matches/ab8543fb/Fulham-L...,https://www.sofascore.com/football/match/fulha...,2025-09-13,1,10,11,10,5.0
5,36,4,https://fbref.com/en/matches/55fbbbe6/Newcastl...,https://www.sofascore.com/football/match/newca...,2025-09-13,1,15,20,15,28.0
6,37,4,None,https://www.sofascore.com/football/match/west-...,2025-09-13,1,19,18,20,10.0
7,38,4,https://fbref.com/en/matches/d8104c65/Brentfor...,https://www.sofascore.com/football/match/brent...,2025-09-13,1,4,7,4,55.0
8,412,4,https://fbref.com/en/matches/a25dff66/Getafe-O...,https://www.sofascore.com/football/match/getaf...,2025-09-13,2,28,36,28,39.0
9,781,3,https://fbref.com/en/matches/5fc86d3a/Cagliari...,https://www.sofascore.com/football/match/cagli...,2025-09-13,3,46,53,47,NaN
